In [1]:
from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint #GpuUsageLogger
from src.settings import RESULT_DIR
from src.conf.config import CONFIG
from src.dl.lightning_model import SegModel, plot_metrics
from src.dl.model_builder import ModelBuilder

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/leos/.local/lib/python3.6/site-packages/torchvision/extension.py:11: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/leos/.local/lib/python3.6/site-packages/torchvision/_C.so'>

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package

# Train a pytorch model

- Any model should work if it's wrapped to lightning SegModel wrapper. e.g. smp models

In [2]:
# get the config file
config = CONFIG

# Set the model to do instance or panoptic segmentation (depends what's in the config.py file)
base_model = ModelBuilder.set_model("FPN", config)

# Insert the model to pytorch lightning framework. (Simplifies the training and other stuff)
lightning_model = SegModel.from_conf(base_model, config)

In [3]:
tt_logger = pl_loggers.TestTubeLogger(
    save_dir=RESULT_DIR,
    name=config.experiment_args.model_name,
    version=config.experiment_args.experiment_version
)

checkpoint_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
)

checkpoint_callback = ModelCheckpoint(
    filepath = str(checkpoint_dir),
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'avg_val_loss',
    mode = 'min',
    prefix = ''
)

if config.training_args.resume_training:   
    last_checkpoint_path = lightning_model.fm.model_checkpoint("last")
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        #callbacks=[GPUStatsMonitor()],
        resume_from_checkpoint=str(last_checkpoint_path),
        profiler=True,
    )
    
else:
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        # callbacks=[GpuUsageLogger()],
        profiler=True,
    )
    

# find the batch size automatically
# new_batch_size = trainer.scale_batch_size(lightning_model)

# Override old batch size
# lightning_model.batch_size = new_batch_size

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:

Checkpoint directory /home/local/leos/Dippa/results/tests/FPN/version_test_consep_FPN exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [4]:
log_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
    / "tf"
)

In [5]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

# Train the model

In [6]:
trainer.fit(lightning_model)

Epoch 6: 100%|█████████▉| 760/761 [04:18<00:00,  2.94it/s, loss=0.229, v_num=_FPN]
Epoch 00006: avg_val_loss  was not in top 1
Epoch 7:  78%|███████▊  | 592/761 [03:05<00:52,  3.20it/s, loss=0.203, v_num=_FPN]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 169/169 [01:07<00:00, 17.00it/s]
Epoch 00007: avg_val_loss  was not in top 1
Epoch 8:  78%|███████▊  | 592/761 [03:12<00:54,  3.08it/s, loss=0.199, v_num=_FPN]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 169/169 [01:05<00:00, 20.41it/s]
Epoch 00008: avg_val_loss  was not in top 1
Epoch 9:  78%|███████▊  | 592/761 [03:07<00:53,  3.15it/s, loss=0.171, v_num=_FPN]
Validating: 0it [00:00, ?it/s]
Validating:  99%|█████████▉| 167/169 [01:04<00:00, 19.15it/s]
Epoch 00009: avg_val_loss reached 0.28972 (best 0.28972), saving model to /home/local/leos/Dippa/results/tests/FPN/version_test_consep_FPN/epoch=9.ckpt as top 1
Epoch 9: 100%|██████████| 761/761 [04:13<00:00,  3.00it/s, loss=0.171, v_num=_FPN]
             

1

In [7]:
# plot_metrics(conf=config, metric='accuracy', scale='linear', save=True)

In [8]:
trainer.test(
    model=lightning_model,
    ckpt_path=lightning_model.fm.model_checkpoint("last").as_posix()
)

Testing:  97%|█████████▋| 37/38 [00:13<00:00,  3.48it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_accuracy': tensor(0.9160, device='cuda:0'),
 'avg_test_loss': tensor(0.2807, device='cuda:0'),
 'loss': tensor(0.2807, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 38/38 [00:13<00:00,  2.76it/s]


[{'avg_test_loss': 0.2807317078113556,
  'avg_test_accuracy': 0.9160057902336121,
  'loss': 0.2807317078113556}]